In [4]:
import pandas as pd

Matplotlib is building the font cache; this may take a moment.


In [ ]:
import re

logfile_path = "openevolve_20250719_072700.log"

with open(logfile_path, "r") as f:
    lines = f.readlines()

island_re = re.compile(
    r"Island (\d+): (\d+) programs, best=([\d.]+), avg=([\d.]+), diversity=([\d.]+), gen=(\d+)(?: \(best: .*\))?"
)
map_elites_re = re.compile(
    r"New MAP-Elites cell occupied: {.*?complexity': (\d+), 'diversity': (\d+)}"
)
iteration_re = re.compile(r"Iteration (\d+): .*completed in ([\d.]+)s")
metrics_re = re.compile(r"Metrics: (.+)")

islands = []
map_elites = []
metrics = []

current_iteration = None
current_time = None

for line in lines:
    # MAP-Elites
    map_match = map_elites_re.search(line)
    if map_match:
        complexity, diversity = map_match.groups()
        map_elites.append({"complexity": int(complexity), "diversity": int(diversity)})

    # Iteration
    iter_match = iteration_re.search(line)
    if iter_match:
        current_iteration, current_time = iter_match.groups()
        current_iteration = int(current_iteration)
        current_time = float(current_time)

    # Metrics
    metric_match = metrics_re.search(line)
    if metric_match and current_iteration is not None:
        metric_data = {}
        metrics_str = metric_match.group(1)
        for m in metrics_str.split(","):
            key, value = m.strip().split("=")
            metric_data[key] = float(value)
        metric_data["iteration"] = current_iteration
        metric_data["time"] = current_time
        metrics.append(metric_data)
        current_iteration = None

    # Island
    island_match = island_re.search(line)
    if island_match:
        island_id, programs, best, avg, diversity, gen = island_match.groups()
        islands.append(
            {
                "Island id": int(island_id),
                "programs amount": int(programs),
                "best": float(best),
                "avg": float(avg),
                "diversity": float(diversity),
                "gen": int(gen),
                "selected": int(programs) > 0,
            }
        )

df_islands = pd.DataFrame(islands)
df_map = pd.DataFrame(map_elites)
df_metrics = pd.DataFrame(metrics)

df_islands.to_csv("df_islands.csv")
df_map.to_csv("df_map.csv")
df_metrics.to_csv("df_metrics.csv")

ISLANDS:
    Island id  programs amount    best     avg  diversity  gen  selected
0           0                1  0.7678  0.7678       0.00    0      True
1           1                0  0.0000  0.0000       0.00    0     False
2           2                0  0.0000  0.0000       0.00    0     False
3           0                2  0.7678  0.6091     311.10    1      True
4           1                0  0.0000  0.0000       0.00    0     False
5           2                0  0.0000  0.0000       0.00    0     False
6           0                3  0.9614  0.7265     263.87    1      True
7           1                0  0.0000  0.0000       0.00    1     False
8           2                0  0.0000  0.0000       0.00    0     False
9           0                3  0.9614  0.7265     263.87    1      True
10          1                2  0.9614  0.8405      90.40    1      True
11          2                0  0.0000  0.0000       0.00    1     False
12          0                3  0.9614  0.

In [6]:
df_islands

,Island id,programs amount,best,avg,diversity,gen,selected
0,0,1,0.7678,0.7678,0.00,0,True
1,1,0,0.0000,0.0000,0.00,0,False
2,2,0,0.0000,0.0000,0.00,0,False
3,0,2,0.7678,0.6091,311.10,1,True
4,1,0,0.0000,0.0000,0.00,0,False
5,2,0,0.0000,0.0000,0.00,0,False
6,0,3,0.9614,0.7265,263.87,1,True
7,1,0,0.0000,0.0000,0.00,1,False
8,2,0,0.0000,0.0000,0.00,0,False
9,0,3,0.9614,0.7265,263.87,1,True


In [7]:
df_map

,complexity,diversity
0,5,0
1,9,5
2,9,0
3,8,0
4,6,0
5,7,0
6,9,3
7,9,4
8,9,9


In [8]:
df_metrics

,runs_successfully,value_score,distance_score,overall_score,iteration,time,standard_deviation_score,speed_score,reliability_score,combined_score,success_rate
0,1.0,0.4890,0.2123,0.1000,0,20.60,NaN,NaN,NaN,NaN,NaN
1,1.0,0.9946,0.9337,0.9923,2,49.24,0.9325,1.0,1.0,0.9614,1.0
2,1.0,0.9018,0.5920,0.9439,4,93.33,0.4840,1.0,1.0,0.7196,1.0
3,1.0,0.6255,0.1408,0.1000,1,118.16,NaN,NaN,NaN,NaN,NaN
4,1.0,0.9711,0.5339,0.9428,5,120.14,0.4352,1.0,1.0,0.7138,1.0
5,1.0,0.9747,0.6595,0.9544,3,174.29,0.4997,1.0,1.0,0.7719,1.0
6,1.0,0.9948,0.9440,0.9941,8,97.68,0.9602,1.0,1.0,0.9706,1.0
7,1.0,0.9919,0.9447,0.9938,7,184.51,0.9549,1.0,1.0,0.9688,1.0
8,1.0,0.9986,0.9654,0.9959,9,171.42,0.9595,1.0,1.0,0.9793,1.0
